In [18]:
import os
import pandas as pd
os.chdir('/home/reffert/DeepAR_InfluenzaForecast')
from PythonFiles.model import preprocessing, train_test_split
from PythonFiles.Configuration import Configuration
from PythonFiles.HpTuning import get_data, objectiveFNN
from ray.air import session
from ray import tune
from datetime import datetime
configuration = Configuration()

In [19]:
configuration.train_start_time = datetime(1999,1,1,0,0,0)
configuration.train_end_time = datetime(2018,9,30,23,0,0)
configuration.test_end_time = datetime(2020,9,30,23,0,0)
# import the data
influenza_df = pd.read_csv("/home/reffert/DeepAR_InfluenzaForecast/Notebooks/DataProcessing/influenza.csv", sep=',')
population_df = pd.read_csv("/home/reffert/DeepAR_InfluenzaForecast/Notebooks/DataProcessing/PopulationVector.csv", sep=',')
neighbourhood_df = pd.read_csv("/home/reffert/DeepAR_InfluenzaForecast/Notebooks/DataProcessing/AdjacentMatrix.csv", sep=',', index_col=0)

data_splits_dict = {}
locations = list(influenza_df.location.unique())
#Process the df into a uniformly spaced df
df = influenza_df.loc[influenza_df.location.isin(locations), ['value', 'location', 'date','week']]
df = preprocessing(configuration, df, check_count=False, output_type="corrected_df")
for location in locations:
    df.loc[df.location == location, "population"] = int(population_df.loc[population_df.Location == location, "2011"].values[0])
    df.loc[df.location == location, locations] = neighbourhood_df.loc[neighbourhood_df.index==location,locations].values[0].astype(int)
data_splits_dict[f"with_features_2001"] = list(train_test_split(configuration, df, True))
train = data_splits_dict["with_features_2001"][0]
test = data_splits_dict["with_features_2001"][1]

In [ ]:
hp_search_space = {
    "num_hidden_dimensions": tune.grid_search([[40, 40, 40]]),
    "context_length":tune.grid_search([104]),
    "epochs":tune.grid_search([200]),
    "prediction_length" : tune.grid_search([4]), 
    "num_batches_per_epoch":tune.grid_search([60]),
    "batch_normalization":tune.grid_search([True]),
    "batch_size":tune.grid_search([124]),
    }

tuner = tune.Tuner(
    tune.with_parameters(objectiveFNN, train=train, test=test, configuration=configuration),
    tune_config=tune.TuneConfig(
        num_samples = 100,
        metric="mean_WIS",
        mode="min",
        max_concurrent_trials=14,
    ),
    param_space=hp_search_space,
)
results = tuner.fit()

print("Best hyperparameters found were: ", results.get_best_result().config)

results_df = results.get_dataframe()
print(results_df)
results_df.to_csv("Tuned_FNN_Hyperparameter_results_06_06.csv")

2023-06-06 10:16:27,330	INFO worker.py:1553 -- Started a local Ray instance.


 67%|██████▋   | 40/60 [00:10<00:05,  3.99it/s, epoch=159/200, avg_epoch_loss=0.795]
